# Final Project

1. Extracting the data sets 

In [1]:
import rasterio
import numpy as np 
import sklearn.ensemble

In [48]:
# import 
# file = '/shared_space/trypy/BioClimp.tif'
# file = '/shared_space/trypy/nlcd2016.tif'
# file = '/shared_space/trypy/sand_mean_rootzone.tif'
# file = '/shared_space/trypy/trydata.h5'
# file = '/shared_space/trypy/clay_mean_rootzone.tif'
# file = '/shared_space/trypy/NED.vrt'
# file = '/shared_space/trypy/om_mean_surface.tif '
# file = '/shared_space/trypy/silt_mean_rootzone.tif'

In [ ]:
# import the bioclim files 


file = '/shared_space/trypy/BioClimp.tif'
#Open up access to the file
fp = rasterio.open(file)

data = np.zeros((30,2707,6362)) # the size of the fp is used 
for i in range(20):
    print(type(i))
    data[i,:,:] = fp.read(i+1)

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [2]:
# dem data 
file = '/shared_space/trypy/dem.tif'
fp = rasterio.open(file)
data[20,:,:] = fp.read(1)

(2707, 6362)
(2707, 6362)


In [3]:
# nlcd206 data 
file = '/shared_space/trypy/nlcd2016.tif'
fp = rasterio.open(file)
data[29,:,:] = fp.read(1)


(2707, 6362)


In [4]:
# sand mean rootzone 
file = '/shared_space/trypy/sand_mean_rootzone.tif'
fp = rasterio.open(file)
data[22,:,:] = fp.read(1)

(2707, 6362)


In [5]:
# clay mean rootzone
file = '/shared_space/trypy/clay_mean_rootzone.tif'
fp = rasterio.open(file)
data[23,:,:] = fp.read(1)

(2707, 6362)


In [6]:
# om mean surface
file = '/shared_space/trypy/om_mean_surface.tif'
fp = rasterio.open(file)
data[24,:,:] = fp.read(1)

(2707, 6362)


In [7]:
# silt mean rootzone 
file = '/shared_space/trypy/silt_mean_rootzone.tif'
fp = rasterio.open(file)
data[21,:,:] = fp.read(1)

(2707, 6362)


In [ ]:
# evi each season
file = '/shared_space/trypy/evi_winter.tif'
fp = rasterio.open(file)
data[25,:,:] = fp.read(1)

file = '/shared_space/trypy/evi_fall.tif'
fp = rasterio.open(file)
data[26,:,:] = fp.read(1)

file = '/shared_space/trypy/evi_spring.tif'
fp = rasterio.open(file)
data[27,:,:] = fp.read(1)

file = '/shared_space/trypy/evi_summer.tif'
fp = rasterio.open(file)
data[28,:,:] = fp.read(1)

In [ ]:
# Import the TRY data
# The bounding box for US
top = 49.3457868 # north lat
left = -124.7844079 # west long
right = -66.9513812 # east long
bottom =  24.7433195 # south lat

# The File 
file = 'trypy/trydata.h5'
fp_try = h5py.File(file,'r')

# Create U.S. Only Mask
lats = fp_try['lat'][:]
lons = fp_try['lon'][:]
mUS = (lats>bottom) & (lats<top) & (lons>left) & (lons<right)

#Remove nodata and create xylem dataset
xyl = fp_try['xylem'][:]
x_mask = (mUS & (xyl!=-9999))
xyl_us = xyl[x_mask]
x_lat = fp_try['lat'][x_mask]
x_lon = fp_try['lon'][x_mask]

#Remove nodata and create root dataset
r = fp_try['root'][:]
r_mask = (mUS & (r!=-9999))
r_us = r[r_mask]
r_lat = fp_try['lat'][r_mask]
r_lon = fp_try['lon'][r_mask]

In [ ]:
# A function to remove invalid data while keeping track of the x,y location
def valid(a,baseshape):
    # a is a 2D array, with dimensions [num features,num samples]
    # baseshape is a tuple for the original spatial shape of the array
    # this function returns a list of valid datapoints with shape [numvalid samples, num features]
    # it also returns 2 vectors, assigning x,y positions to each sample
    x1=np.ones(baseshape)
    y1=np.ones(baseshape)
    n = x1.size
    nin = np.shape(a)[1]
    for r in range(baseshape[0]):
        x1[r,:]=r
    for c in range(baseshape[1]):
        y1[:,c]=c
    x1 = np.reshape(x1,(n))
    y1 = np.reshape(y1,(n))
    mask = np.ones(np.shape(x1), dtype=bool)
    for e in range(nin):
        mask = mask & (~np.isnan(a[:,e]) & (a[:,e]>-2500) & (a[:,e]<10**10))
    c0 = a[mask,0]
    l = np.shape(c0)[0]
    aout=np.ones((l,nin))                               
    for e in range(nin):
        aout[:,e]=a[mask,e]
    x1 = x1[mask]
    y1 = y1[mask]
    return aout, x1, y1

In [ ]:
k= 20+10
#flatten to 2D for PCA and RF
flat = np.reshape(data,(k,2707*6362))

#Remove invalid data in prep for PCA and RF
val_data, x_pos,y_pos = valid(np.transpose(flat),(2707,6362))

#Remove NLCD since it is categorical and thus not well suited to PCA
Xsites = val_data[:,0:29]

#standardization
Xsites_std = (Xsites - np.mean(Xsites,axis=0))/np.std(Xsites,axis=0)

In [ ]:
# Code to determine optimal number of components
"""
#optimal number of components
LIST=[]
for i in range(k-1):
    pca = sklearn.decomposition.PCA(n_components=i)
    pca.fit(Xsites_std)
    tmp= np.sum(pca.explained_variance_ratio_)
    LIST.append(tmp)
    print(str(i)+':'+str(tmp))
    
x= np.linspace(1,k-1,k-1)
plt.plot(x,LIST)
plt.title('optimal PCA number')
plt.xlabel('number of principle components')
plt.ylabel('expalined variance ratio')
plt.show()
"""

In [ ]:
# the optimal PCA
pca = sklearn.decomposition.PCA(n_components=15)
pca.fit(Xsites_std)
Xsites_std_pca = pca.transform(Xsites_std)
print('Done0')

In [ ]:
#Random Forest -- PCA, No Removal of outliers
''' Prep for Random Forest '''
X_x = [[] for i in range(15)]
X_r = [[] for i in range(15)]
y_x = []
y_r = []
# Extract the feature values at each location of our plant trait samples xylem
for x in range(948):
    pos = fp.index(x_lon[x],x_lat[x])
    pos_linear = np.where((x_pos == pos[0])&(y_pos == pos[1]))
    if (pos_linear[0].size == 0):
        continue
    for i in range(15):
         X_x[i].append(float(Xsites_std_pca[pos_linear[0][0],i]))
    y_x.append(xyl_us[x])
    
# Do the Same for Rooting depth
for x in range(1573):
    pos = fp.index(r_lon[x],r_lat[x])
    pos_linear = np.where((x_pos == pos[0])&(y_pos == pos[1]))
    if (pos_linear[0].size == 0):
        continue
    for i in range(15):
        X_r[i].append(float(Xsites_std_pca[pos_linear[0],i]))
    y_r.append(r_us[x])
    
X_x = np.transpose(np.array(X_x))
X_r = np.transpose(np.array(X_r))

clf = sklearn.ensemble.RandomForestRegressor(random_state=1,n_estimators=100,oob_score=True)
clf.fit(X_x,y_x)
xscore = clf.oob_score_
xpredict = clf.predict(Xsites_std_pca)

clf = sklearn.ensemble.RandomForestRegressor(random_state=1,n_estimators=100,oob_score=True)
clf.fit(X_r,y_r)
rscore = clf.oob_score_
rpredict = clf.predict(Xsites_std_pca)

# Map the predictions
r_map = np.ones((2707,6362))*float('nan')
for i in range(x_pos.size):
    r_map[int(x_pos[i]),int(y_pos[i])]=rpredict[i]

x_map = np.ones((2707,6362))*float('nan')
for i in range(x_pos.size):
    x_map[int(x_pos[i]),int(y_pos[i])]=xpredict[i]

In [ ]:
# %% Random Forest -- PCA, with Removal of outliers
''' Prep for Random Forest '''
X_r = [[] for i in range(15)]
y_r = []
# Extract the feature values at each location for Rooting depth
for x in range(1573):
    pos = fp.index(r_lon[x],r_lat[x])
    pos_linear = np.where((x_pos == pos[0])&(y_pos == pos[1]))
    if (pos_linear[0].size == 0):
        continue
    if (r_us[x]>np.percentile(r_us,98)):
        continue
    for i in range(15):
        X_r[i].append(float(Xsites_std_pca[pos_linear[0],i]))
    y_r.append(r_us[x])

X_r = np.transpose(np.array(X_r))

clf = sklearn.ensemble.RandomForestRegressor(random_state=1,n_estimators=100,oob_score=True)
clf.fit(X_r,y_r)
rscore96 = clf.oob_score_
rpredict = clf.predict(Xsites_std_pca)

# Map the predictions
r_map96 = np.ones((2707,6362))*float('nan')
for i in range(x_pos.size):
    r_map96[int(x_pos[i]),int(y_pos[i])]=rpredict[i]

In [ ]:
# Random Forest --No PCA with Removal of outliers
''' Prep for Random Forest '''
X_x = [[] for i in range(30)]
X_r = [[] for i in range(30)]
y_x = []
y_r = []
# Extract the feature values at each location of our plant trait samples xylem
for x in range(948):
    pos = fp.index(x_lon[x],x_lat[x])
    pos_linear = np.where((x_pos == pos[0])&(y_pos == pos[1]))
    if (pos_linear[0].size == 0):
        continue
    for i in range(30):
         X_x[i].append(float(val_data[pos_linear[0][0],i]))
    y_x.append(xyl_us[x])
    
# Do the Same for Rooting depth
for x in range(1573):
    pos = fp.index(r_lon[x],r_lat[x])
    pos_linear = np.where((x_pos == pos[0])&(y_pos == pos[1]))
    if (pos_linear[0].size == 0):
        continue
    if (r_us[x]>np.percentile(r_us,98)):
        continue
    for i in range(30):
        X_r[i].append(float(val_data[pos_linear[0],i]))
    y_r.append(r_us[x])
    
X_x = np.transpose(np.array(X_x))
X_r = np.transpose(np.array(X_r))

clf = sklearn.ensemble.RandomForestRegressor(random_state=1,n_estimators=100,oob_score=True)
clf.fit(X_x,y_x)
xscore_nopca = clf.oob_score_
xpredict = clf.predict(val_data)

clf = sklearn.ensemble.RandomForestRegressor(random_state=1,n_estimators=100,oob_score=True)
clf.fit(X_r,y_r)
rscore_nopca = clf.oob_score_
rpredict = clf.predict(val_data)

# Map the predictions
r_map_nopca = np.ones((2707,6362))*float('nan')
for i in range(x_pos.size):
    r_map[int(x_pos[i]),int(y_pos[i])]=rpredict[i]

x_map_nopca = np.ones((2707,6362))*float('nan')
for i in range(x_pos.size):
    x_map[int(x_pos[i]),int(y_pos[i])]=xpredict[i]
    

In [ ]:
#Save Tif: PCA without removal Rooting Depth
#fp is the fp from the first few cell blocks
with rasterio.Env():
    profile = fp.profile
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open('trypy/rooting_depth_pca.tif', 'w', **profile) as dst:
        dst.write(r_map.astype(rasterio.float32), 1)

In [ ]:
#Save Tif: PCA xylem
with rasterio.Env():
    profile = fp.profile
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open('trypy/xylem.tif', 'w', **profile) as dst:
        dst.write(x_map.astype(rasterio.float32), 1)

In [ ]:
#Save Tif: PCA with removal Rooting Depth
with rasterio.Env():
    profile = fp.profile
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open('trypy/rooting_depth_pca_removal.tif', 'w', **profile) as dst:
        dst.write(r_map96.astype(rasterio.float32), 1)

In [ ]:
#Save Tif: No PCA Rooting Depth
with rasterio.Env():
    profile = fp.profile
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open('trypy/rooting_depth_nopca_removal.tif', 'w', **profile) as dst:
        dst.write(r_map_nopca.astype(rasterio.float32), 1)

In [ ]:
#Save Tif: No PCA Xylem
with rasterio.Env():
    profile = fp.profile
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open('trypy/xylem_nopca.tif', 'w', **profile) as dst:
        dst.write(x_map_nopca.astype(rasterio.float32), 1)

SUMMARY OF OOB SCORES
Rooting Depth:
    PCA without removal of outliers      .594
    PCA with removal of outliers         .642
    No PCA with removal of outliers      .652
Xylem
    PCA (without removal of outliers)    .474
    no PCA (without removal of outliers) .519